# NORDic DR: Adaptive sampling with bandit algorithms

The drug signatures and patients/control phenotypes were computed in notebook ``Drug_Simulation.ipynb``.

## Library Import

In [1]:
import NORDic
! pip freeze | grep "NORDic"

NORDic==2.4.2


In [2]:
import pandas as pd
import os
import random
import numpy as np
from multiprocessing import cpu_count

from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [3]:
from NORDic.UTILS.utils_state import binarize_experiments
from NORDic.NORDic_DR.functions import adaptive_testing

## Initialization (states, drug signatures, scoring function)

In [5]:
file_folder="Breast_Cancer/"
seed_number=0
solution_fname=file_folder+"solution_connected.bnet"
file_folder_DS=file_folder+"DS/"

In [6]:
signature_fname=file_folder_DS+"signatures_drugs_binarized.csv"
path_to_initial_states=file_folder+"initial_states.csv"
## grid search computed in Drug_Simulation notebook (optional)
path_to_scores=file_folder_DS+"scores.csv"

assert os.path.exists(signature_fname) ## signatures
assert os.path.exists(path_to_initial_states) ## phenotypes

In [7]:
## drug signatures
signatures = pd.read_csv(signature_fname, index_col=0)
signatures = signatures.T.dropna(how="all").T

## ground truth from GDSC database
zIC50 = pd.read_csv(file_folder_DS+"Cellline_downloadWed Apr 19 15 48 11 2023.csv", 
                  index_col=1)[["Z Score"]]
zIC50 = -zIC50.loc[~zIC50.index.duplicated()]
zIC50.columns = ["Ground Truth"]

## patient phenotypes
metadata = pd.read_csv(file_folder+"metadata_GSE42568.tsv", index_col=0, sep=" \t", engine="python", header=None)
df = pd.read_csv(path_to_initial_states, index_col=0)
metadata_patients = metadata.loc[[i for i in metadata.index if ("cancer" in metadata.loc[i][1])]]
metadata_controls = metadata.loc[[i for i in metadata.index if ("cancer" not in metadata.loc[i][1])]]
patients = df[[i for i in metadata_patients.index if (i in df.columns)]]
controls = df[[i for i in metadata_controls.index if (i in df.columns)]]
phenotypes = pd.concat((patients, controls), axis=1)
phenotypes.loc["annotation"] = [2]*patients.shape[1]+[1]*controls.shape[1]
samples = list(phenotypes.loc["annotation"])
## Same approach as in Drug_Simulation.ipynb
dfsdata = phenotypes.loc[[idx for idx in phenotypes.index if (idx!="annotation")]]
bin_mat = binarize_experiments(dfsdata, thres=0.5, method="binary")
bin_mat = bin_mat.dropna()
states = bin_mat.astype(int)
outliers_ctrls = [105,106,107]
samples = [samples[i] for i in range(len(samples)) if (i not in outliers_ctrls)]
states = states[[states.columns[i] for i in range(len(samples)) if (i not in outliers_ctrls)]]

In [8]:
assert os.path.exists(path_to_scores)
scores = pd.read_csv(path_to_scores, index_col=0) 

## use score matrix stored in memory to decrease runtime (for demonstration purposes)
rewards_fname = file_folder_DS+"rewards.csv"
if (not os.path.exists(rewards_fname)):
    scores.to_csv(rewards_fname)

args = (scores.shape[0], scores.shape[0], scores.shape[1]*scores.shape[0])
print("%d patients, %d drugs = %d simulations in a grid search" % args)

scores_mean = pd.DataFrame(scores.mean(axis=0), columns=["Score"]) ## average drug score across all patients
scores_mean.sort_values(by="Score", ascending=False).head()

104 patients, 104 drugs = 6032 simulations in a grid search


,Score
Doramapimod,0.530612
Temsirolimus,0.526310
Camptothecin,0.516994
Cediranib,0.516183
Lapatinib,0.515306


In [9]:
## Scoring function for drugs
def score(attrs):
    idxs = [g for g in list(attrs.index) if (g in list(states.index))]
    dists = cdist(attrs.loc[idxs].values.T, states.loc[idxs].values.T, metric='cityblock')
    scores_attrs = []
    for ia in range(attrs.shape[1]):
        min_dist = np.min(dists[ia,:])/float(len(idxs))
        argmin_dist = [samples[y] for x, y in list(np.argwhere(dists[ia,:] == min_dist))]
        nctrl, npat = [sum([x for x in argmin_dist if (x==v)]) for v in [1,2]]
        scores_attrs.append(min_dist*(-1)**int(npat>nctrl))
    return np.array(scores_attrs)

## Initialization of drug features (as PCA components) and call to the bandit algorithm

In [33]:
random.seed(seed_number)
np.random.seed(seed_number)

n_components=6
pca, scaler = PCA(n_components=n_components), StandardScaler()
X = targets1.values.T
X = scaler.fit_transform(X)
pca.fit(X)
print("Explained variance %.1f" % np.cumsum(pca.explained_variance_)[-1])
X_pca = pca.transform(X)
features1 = pd.DataFrame(X_pca.T, index=["PC%d" % (i+1) for i in range(n_components)], columns=targets1.columns)

BANDIT_args = {
    'bandit': 'LinGapE', 
    #type of algorithm, (greedy) LinGapE is faster but more prone to errors 
    #(assumes that the model is linear)
    'seed': seed_number,
    'delta': 0.1, #error rate
    'nsimu': 500, #number of repeats
    'm': 1, #number of recommendations to make
    'c': 0., #nonnegative parameter to tune for MisLid (if the model is linear, set to 0
    ## To speed up the algorithm, decrease c
    ## To ensure correctness of the recommendation, increase c
    'sigma': 1,
    'beta': "heuristic",
    'epsilon': 0.1,
    'tracking_type': "D",
    'gain_type': "empirical",
    'learner': "AdaHedge"
}

njobs=max(1,cpu_count()-2)
SIMU_params = {
    'nb_sims': 100,
    'rates': "fully_asynchronous",
    'thread_count': njobs,
    'depth': "constant_unitary",
}

## target vectors for the simulation
targets=signatures
idxs = [g for g in targets.index if (g in patients.index)]
patients1, targets1 = patients.loc[idxs], targets.loc[idxs]
targets1[targets1 == 0] = -1
targets1 = targets1.fillna(0)

## if no prior grid search has been performed, one might set reward_fname=None
recommendation = adaptive_testing(solution_fname, features1, targets1, score, patients1, 
       SIMU_params, BANDIT_args, reward_fname=rewards_fname, quiet=False).T

Explained variance 65.7
<NORDic DR> Avg. #samples = 5340, avg. runtime 29.527869988918305 sec (over 500 iterations)


In [29]:
## average recommendation across all 500 iterations
rec = recommendation.join(scores_mean)
rec.join(zIC50).sort_values(by="Frequency", ascending=False).head()

,Frequency,Score,Ground Truth
Temsirolimus,1.0,0.526310,0.657906
Afatinib,0.0,0.331633,-1.106644
Sorafenib,0.0,0.464089,0.590431
Mirin,0.0,0.387755,-0.349519
Mitoxantrone,0.0,0.418367,-1.162713
